In [1]:
import numpy as np
import tensorflow as tf
from my_utils import load_mnist_data
from matplotlib import pyplot as plt
from time import time

In [2]:
train_images, train_labels, test_images, test_labels = load_mnist_data()
train_images = np.multiply(train_images, 1.0 / 255.0)
test_images = np.multiply(test_images, 1.0 / 255.0)

In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [4]:
tf.reset_default_graph()

In [5]:
learning_rate = 0.001
total_epoch = 10
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [6]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

In [7]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

In [8]:
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = 60000 // batch_size
indices = np.arange(0, 60000)

for epoch in range(total_epoch):
    total_cost = 0
    np.random.shuffle(indices)
    xs = train_images[indices, :]
    ys = train_labels[indices, :]
    
    for i in range(total_batch):
        batch_xs = xs[i * batch_size : (i+1) * batch_size, :].reshape([batch_size, n_step, n_input])
        batch_ys = ys[i * batch_size : (i+1) * batch_size, :]
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys})
        
        total_cost += cost_val
    
    print(f'Epoch {epoch}: {total_cost / total_batch}')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('accuracy:', sess.run(accuracy, feed_dict={X:test_images.reshape(10000, n_step, n_input), Y:test_labels}))

Epoch 0: 0.5892516097579247
Epoch 1: 0.24363366848765275
Epoch 2: 0.18188388905145675
Epoch 3: 0.1478098920490752
Epoch 4: 0.1359592511231064
Epoch 5: 0.12405933911362901
Epoch 6: 0.10902699009658626
Epoch 7: 0.09918927849055482
Epoch 8: 0.09753011615994649
Epoch 9: 0.09934697714614339
accuracy: 0.9637
